# Tech Challenge - Fine-tuning para Produtos Amazon (GPU Colab)

**Objetivo**: Executar fine-tuning de um foundation model usando o dataset AmazonTitles-1.3MM para gerar descrições de produtos baseadas em títulos.

**Dataset**: Utilizaremos o arquivo `trn.json.gz` que contém títulos e descrições de produtos da Amazon.

**Modelo Escolhido**: TinyLlama 1.1B com Unsloth para otimização de treinamento.

---

## 📋 Índice
1. [Configuração Inicial](#1-configuracao-inicial)
2. [Exploração dos Dados](#2-exploracao-dos-dados)
3. [Preparação do Dataset](#3-preparacao-do-dataset)
4. [Teste do Modelo Base](#4-teste-do-modelo-base)
5. [Fine-tuning](#5-fine-tuning)
6. [Teste do Modelo Treinado](#6-teste-do-modelo-treinado)
7. [Demonstração Interativa](#7-demonstracao-interativa)

---

## 1. Configuração Inicial

### 1.1 Montagem do Google Drive
Primeiro, vamos montar o Google Drive para acessar e salvar nossos arquivos.

In [ ]:
from google.colab import drive
import os

# Monta o Google Drive
drive.mount('/content/drive')

# Define o diretório de trabalho (usando o mesmo diretório onde está o arquivo de dados)
WORK_DIR = '/content/drive/MyDrive/FineTunning/TechChallenge03'
os.makedirs(WORK_DIR, exist_ok=True)

print(f"✅ Google Drive montado com sucesso!")
print(f"📁 Diretório de trabalho: {WORK_DIR}")

# Verifica se o diretório existe e lista os arquivos
if os.path.exists(WORK_DIR):
    files_in_dir = os.listdir(WORK_DIR)
    print(f"📋 Arquivos no diretório: {files_in_dir}")
else:
    print(f"⚠️ Diretório não existe, será criado: {WORK_DIR}")

### 1.2 Instalação das Dependências

Instalamos as bibliotecas necessárias:
- **Unsloth**: Otimização para fine-tuning eficiente
- **Transformers**: Biblioteca principal para modelos de linguagem
- **Datasets**: Para manipulação de datasets
- **TRL**: Para treinamento de modelos de linguagem

In [ ]:
# Instalação das dependências principais
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets torch

print("✅ Todas as dependências foram instaladas com sucesso!")

### 1.3 Importação das Bibliotecas e Configurações Iniciais

In [ ]:
# Imports necessários
import json
import gzip
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset
import torch
from transformers import TrainingArguments, TextStreamer
from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotecas importadas com sucesso!")
print(f"🔥 CUDA disponível: {torch.cuda.is_available()}")
print(f"💾 GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Não disponível'}")

### 1.4 Configurações do Modelo e Treinamento

In [ ]:
# Configurações principais
CONFIG = {
    # Configurações do modelo
    'model_name': "unsloth/tinyllama-bnb-4bit",
    'max_seq_length': 1024,
    'dtype': None,  # Será determinado automaticamente
    'load_in_4bit': True,
    
    # Configurações do dataset
    'data_file': '/content/drive/MyDrive/FineTunning/TechChallenge03/trn.json.gz',
    'sample_size': 50000,  # Número de amostras para treinamento (otimizado para Colab)
    'test_size': 5000,  # Número de amostras para teste
    
    # Configurações do fine-tuning
    'lora_r': 32,
    'lora_alpha': 16,
    'lora_dropout': 0,
    'max_steps': 100,  # Ajuste conforme necessário
    'learning_rate': 2e-4,
    'batch_size': 4,
    'gradient_accumulation_steps': 2,
    
    # Caminhos - usando o mesmo diretório base
    'base_dir': '/content/drive/MyDrive/FineTunning/TechChallenge03',
    'output_dir': '/content/drive/MyDrive/FineTunning/TechChallenge03/outputs',
    'model_save_path': '/content/drive/MyDrive/FineTunning/TechChallenge03/amazon_model',
}

# Criação dos diretórios
os.makedirs(CONFIG['base_dir'], exist_ok=True)
os.makedirs(CONFIG['output_dir'], exist_ok=True)
os.makedirs(CONFIG['model_save_path'], exist_ok=True)

print("⚙️ Configurações definidas:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## 2. Exploração dos Dados

### 2.1 Upload do Arquivo de Dados

Primeiro, você precisa fazer upload do arquivo `trn.json.gz` para o Colab.
Execute a célula abaixo e faça upload do arquivo quando solicitado.

In [ ]:
import os

# Define o caminho para o arquivo no Google Drive
DATA_FILE_PATH = '/content/drive/MyDrive/FineTunning/TechChallenge03/trn.json.gz'

# Verifica se o arquivo existe
if os.path.exists(DATA_FILE_PATH):
    print("✅ Arquivo trn.json.gz encontrado no Google Drive!")
    print(f"📁 Caminho: {DATA_FILE_PATH}")
    
    # Verifica o tamanho do arquivo
    file_size = os.path.getsize(DATA_FILE_PATH)
    print(f"📊 Tamanho do arquivo: {file_size / (1024*1024):.1f} MB")
else:
    print("❌ Arquivo não encontrado no caminho especificado.")
    print(f"❌ Caminho verificado: {DATA_FILE_PATH}")
    print("💡 Certifique-se de que o arquivo trn.json.gz está no diretório correto do Google Drive.")
    DATA_FILE_PATH = None

### 2.2 Carregamento dos Dados

In [ ]:
def load_data_sample(file_path, sample_size=100000):
    """
    Carrega uma amostra dos dados do arquivo JSON comprimido
    """
    data = []
    print(f"🔄 Carregando dados de {file_path}...")
    
    try:
        with gzip.open(file_path, 'rt', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i >= sample_size:
                    break
                    
                try:
                    json_obj = json.loads(line.strip())
                    data.append(json_obj)
                except json.JSONDecodeError:
                    continue
                    
                # Progress indicator
                if (i + 1) % 10000 == 0:
                    print(f"📊 Carregados {i + 1} registros...")
    
    except Exception as e:
        print(f"❌ Erro ao carregar dados: {str(e)}")
        return []
    
    print(f"✅ Carregamento concluído! Total de registros: {len(data)}")
    return data

# Carrega uma amostra dos dados para exploração
if DATA_FILE_PATH and os.path.exists(DATA_FILE_PATH):
    sample_data = load_data_sample(DATA_FILE_PATH, sample_size=50000)
    print(f"📋 Amostra carregada com {len(sample_data)} registros")
else:
    print("❌ Não foi possível carregar os dados. Verifique o arquivo.")

### 2.3 Análise Exploratória dos Dados

In [ ]:
if sample_data:
    print("🔍 Análise dos dados carregados:\n")
    
    # Estrutura dos dados
    print("📋 Estrutura dos dados:")
    if sample_data:
        print(f"  Primeiro registro: {sample_data[0]}")
        print(f"  Chaves disponíveis: {list(sample_data[0].keys())}")
    
    # Converte para DataFrame para análise
    df = pd.DataFrame(sample_data)
    print(f"\n📊 Estatísticas básicas:")
    print(f"  Total de registros: {len(df)}")
    print(f"  Colunas: {list(df.columns)}")
    
    # Análise dos títulos
    if 'title' in df.columns:
        title_lengths = df['title'].str.len()
        print(f"\n📏 Análise dos títulos:")
        print(f"  Comprimento médio: {title_lengths.mean():.1f} caracteres")
        print(f"  Comprimento mediano: {title_lengths.median():.1f} caracteres")
        print(f"  Mínimo: {title_lengths.min()} caracteres")
        print(f"  Máximo: {title_lengths.max()} caracteres")
    
    # Análise das descrições
    if 'content' in df.columns:
        content_lengths = df['content'].str.len()
        print(f"\n📝 Análise das descrições:")
        print(f"  Comprimento médio: {content_lengths.mean():.1f} caracteres")
        print(f"  Comprimento mediano: {content_lengths.median():.1f} caracteres")
        print(f"  Mínimo: {content_lengths.min()} caracteres")
        print(f"  Máximo: {content_lengths.max()} caracteres")
    
    # Mostra alguns exemplos
    print(f"\n📝 Exemplos de dados:")
    for i in range(min(3, len(df))):
        print(f"\n--- Exemplo {i+1} ---")
        for col in df.columns:
            content = str(df[col].iloc[i])
            if len(content) > 100:
                content = content[:100] + "..."
            print(f"  {col}: {content}")
else:
    print("❌ Nenhum dado disponível para análise.")

## 3. Preparação do Dataset

### 3.1 Tratamento e Limpeza dos Dados

Vamos implementar um sistema robusto de limpeza dos dados que já provou ser eficaz, garantindo que apenas dados de alta qualidade sejam usados no treinamento.

In [ ]:
import re
from collections import Counter

def is_valid_text(text, min_length=10, max_length=500):
    """Verifica se o texto é válido para treinamento"""
    if not text or not isinstance(text, str):
        return False
    
    text = text.strip()
    if len(text) < min_length or len(text) > max_length:
        return False
    
    # Verifica se tem conteúdo significativo (não apenas símbolos)
    if len(re.sub(r'[^a-zA-Z0-9\s]', '', text).strip()) < min_length // 2:
        return False
    
    return True

def clean_text(text):
    """Limpa e padroniza o texto"""
    if not text:
        return ""
    
    # Remove caracteres especiais excessivos
    text = re.sub(r'[^\w\s\-.,!?()&]', ' ', text)
    
    # Remove espaços múltiplos
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def prepare_training_data(data, sample_size=50000):
    """
    Prepara os dados para treinamento com limpeza avançada
    """
    print(f"🔄 Processando dados para treinamento...")
    
    processed_data = []
    rejected_count = 0
    
    for i, item in enumerate(data[:sample_size]):
        try:
            # Extrai título e conteúdo
            title = item.get('title', '').strip()
            content = item.get('content', '').strip()
            
            # Limpa os textos
            title_clean = clean_text(title)
            content_clean = clean_text(content)
            
            # Valida qualidade
            if (is_valid_text(title_clean, min_length=5, max_length=200) and 
                is_valid_text(content_clean, min_length=20, max_length=800)):
                
                # Formata no padrão de instruction-following
                formatted_text = f"""### Instruction:
Generate a detailed product description based on the following title.

### Input:
{title_clean}

### Response:
{content_clean}"""
                
                processed_data.append({
                    'text': formatted_text,
                    'title': title_clean,
                    'content': content_clean
                })
            else:
                rejected_count += 1
        
        except Exception as e:
            rejected_count += 1
            continue
        
        # Progress indicator
        if (i + 1) % 10000 == 0:
            approval_rate = (len(processed_data) / (i + 1)) * 100
            print(f"📊 Processados {i + 1} | Aprovados: {len(processed_data)} ({approval_rate:.1f}%)")
    
    final_approval_rate = (len(processed_data) / len(data[:sample_size])) * 100
    print(f"\n✅ Processamento concluído!")
    print(f"📈 Taxa de aprovação final: {final_approval_rate:.1f}%")
    print(f"✅ Dados aprovados: {len(processed_data)}")
    print(f"❌ Dados rejeitados: {rejected_count}")
    
    return processed_data

# Processa os dados
if sample_data:
    training_data = prepare_training_data(sample_data, CONFIG['sample_size'])
    print(f"\n🎯 Dataset final: {len(training_data)} amostras prontas para treinamento")
else:
    print("❌ Nenhum dado disponível para processamento.")

### 3.2 Criação do Dataset do Hugging Face

In [ ]:
if training_data:
    # Converte para formato do Hugging Face
    dataset_dict = {'text': [item['text'] for item in training_data]}
    dataset = Dataset.from_dict(dataset_dict)
    
    print(f"✅ Dataset criado com sucesso!")
    print(f"📊 Total de amostras: {len(dataset)}")
    print(f"📏 Comprimento médio do texto: {np.mean([len(text) for text in dataset_dict['text']]):.0f} caracteres")
    
    # Mostra exemplo do formato
    print(f"\n📝 Exemplo do formato de treinamento:")
    print("=" * 60)
    print(dataset[0]['text'][:500] + "..." if len(dataset[0]['text']) > 500 else dataset[0]['text'])
    print("=" * 60)
    
else:
    print("❌ Não foi possível criar o dataset.")

## 4. Teste do Modelo Base

### 4.1 Carregamento do Modelo TinyLlama

In [ ]:
# Carrega o modelo base TinyLlama
print("🔄 Carregando modelo TinyLlama...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=CONFIG['model_name'],
    max_seq_length=CONFIG['max_seq_length'],
    dtype=CONFIG['dtype'],
    load_in_4bit=CONFIG['load_in_4bit'],
)

print("✅ Modelo TinyLlama carregado com sucesso!")
print(f"📊 Modelo: {CONFIG['model_name']}")
print(f"📏 Comprimento máximo de sequência: {CONFIG['max_seq_length']}")
print(f"🔧 Quantização 4-bit: {CONFIG['load_in_4bit']}")

# Informações sobre o modelo
print(f"\n📈 Estatísticas do modelo:")
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"  Parâmetros totais: {total_params:,}")
print(f"  Parâmetros treináveis: {trainable_params:,}")
print(f"  Tamanho do vocabulário: {len(tokenizer)}")

### 4.2 Teste do Modelo Base

In [ ]:
# Função para testar o modelo
def test_model(model, tokenizer, prompt, max_new_tokens=150):
    """Testa o modelo com um prompt"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # Configurações de geração
    generation_config = {
        'max_new_tokens': max_new_tokens,
        'temperature': 0.7,
        'do_sample': True,
        'top_p': 0.9,
        'pad_token_id': tokenizer.eos_token_id
    }
    
    with torch.no_grad():
        outputs = model.generate(**inputs, **generation_config)
    
    # Decodifica apenas os tokens gerados (não inclui o prompt)
    generated_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return generated_text.strip()

# Teste com um exemplo
test_prompt = """### Instruction:
Generate a detailed product description based on the following title.

### Input:
Wireless Bluetooth Headphones with Noise Cancellation

### Response:"""

print("🧪 Testando modelo base (antes do fine-tuning):")
print("=" * 60)
print(f"Prompt: {test_prompt}")
print("\nResposta do modelo base:")
print("-" * 40)

try:
    base_response = test_model(model, tokenizer, test_prompt)
    print(base_response)
except Exception as e:
    print(f"❌ Erro no teste: {str(e)}")

print("=" * 60)

## 5. Fine-tuning

### 5.1 Configuração do LoRA

In [ ]:
# Configuração do LoRA (Low-Rank Adaptation)
print("🔧 Configurando LoRA para fine-tuning...")

model = FastLanguageModel.get_peft_model(
    model,
    r=CONFIG['lora_r'],
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=CONFIG['lora_alpha'],
    lora_dropout=CONFIG['lora_dropout'],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

print("✅ LoRA configurado com sucesso!")

# Mostra estatísticas dos parâmetros treináveis
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n📊 Estatísticas após configuração LoRA:")
print(f"  Parâmetros totais: {total_params:,}")
print(f"  Parâmetros treináveis: {trainable_params:,}")
print(f"  Percentual treinável: {100 * trainable_params / total_params:.2f}%")
print(f"🎯 LoRA configurado com r={CONFIG['lora_r']}, alpha={CONFIG['lora_alpha']}")

### 5.2 Configuração do Treinamento

In [ ]:
# Configuração dos argumentos de treinamento
training_args = TrainingArguments(
    per_device_train_batch_size=CONFIG['batch_size'],
    gradient_accumulation_steps=CONFIG['gradient_accumulation_steps'],
    warmup_steps=5,
    max_steps=CONFIG['max_steps'],
    learning_rate=CONFIG['learning_rate'],
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir=CONFIG['output_dir'],
    save_steps=25,
    save_total_limit=3,
    dataloader_num_workers=0,
    remove_unused_columns=False,
)

print("⚙️ Argumentos de treinamento configurados:")
print(f"  Batch size: {CONFIG['batch_size']}")
print(f"  Accumulation steps: {CONFIG['gradient_accumulation_steps']}")
print(f"  Learning rate: {CONFIG['learning_rate']}")
print(f"  Max steps: {CONFIG['max_steps']}")
print(f"  Precision: {'BF16' if is_bfloat16_supported() else 'FP16'}")
print(f"  Output dir: {CONFIG['output_dir']}")

# Configuração do trainer
if 'dataset' in locals():
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset,
        dataset_text_field="text",
        max_seq_length=CONFIG['max_seq_length'],
        dataset_num_proc=2,
        packing=False,
        args=training_args,
    )
    
    print("✅ Trainer configurado com sucesso!")
    print(f"📊 Dataset de treinamento: {len(dataset)} amostras")
else:
    print("❌ Dataset não disponível para configuração do trainer.")

### 5.3 Execução do Fine-tuning

In [ ]:
# Execução do treinamento
print("🚀 Iniciando fine-tuning...")
print("⏱️ Isso pode levar alguns minutos dependendo da configuração da GPU...")

if 'trainer' in locals():
    try:
        # Mostra estatísticas da GPU antes do treinamento
        if torch.cuda.is_available():
            print(f"🔥 GPU: {torch.cuda.get_device_name(0)}")
            print(f"💾 Memória GPU total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
        
        # Executa o treinamento
        trainer_stats = trainer.train()
        
        print("✅ Fine-tuning concluído com sucesso!")
        print(f"📊 Estatísticas do treinamento:")
        print(f"  Steps totais: {trainer_stats.global_step}")
        print(f"  Loss final: {trainer_stats.training_loss:.4f}")
        print(f"  Tempo total: {trainer_stats.metrics.get('train_runtime', 0):.2f} segundos")
        
    except Exception as e:
        print(f"❌ Erro durante o treinamento: {str(e)}")
        print("💡 Dica: Verifique se há memória GPU suficiente ou reduza o batch size.")
else:
    print("❌ Trainer não configurado. Execute as células anteriores primeiro.")

## 6. Teste do Modelo Treinado

### 6.1 Salvar o Modelo

In [ ]:
# Salva o modelo treinado
print("💾 Salvando o modelo treinado...")

try:
    # Salva o modelo LoRA
    model.save_pretrained(CONFIG['model_save_path'])
    tokenizer.save_pretrained(CONFIG['model_save_path'])
    
    print(f"✅ Modelo salvo com sucesso em: {CONFIG['model_save_path']}")
    
    # Lista os arquivos salvos
    import os
    saved_files = os.listdir(CONFIG['model_save_path'])
    print(f"📁 Arquivos salvos: {saved_files}")
    
except Exception as e:
    print(f"❌ Erro ao salvar o modelo: {str(e)}")

### 6.2 Teste do Modelo Fine-tunado

In [ ]:
# Testa o modelo após o fine-tuning
test_prompts = [
    "Wireless Bluetooth Headphones with Noise Cancellation",
    "Professional Gaming Keyboard with RGB Lighting",
    "Stainless Steel Water Bottle 32oz",
    "Organic Cotton T-Shirt for Men",
    "Smart Fitness Tracker with Heart Rate Monitor"
]

print("🧪 Testando modelo após fine-tuning:")
print("=" * 70)

for i, title in enumerate(test_prompts, 1):
    test_prompt = f"""### Instruction:
Generate a detailed product description based on the following title.

### Input:
{title}

### Response:"""
    
    print(f"\n🔹 Teste {i}: {title}")
    print("-" * 50)
    
    try:
        response = test_model(model, tokenizer, test_prompt, max_new_tokens=200)
        print(response)
    except Exception as e:
        print(f"❌ Erro no teste: {str(e)}")
    
    print("-" * 50)

print("=" * 70)

## 7. Demonstração Interativa

### 7.1 Interface de Teste Interativo

In [ ]:
def generate_product_description(title, max_length=200, temperature=0.7):
    """
    Gera descrição de produto baseada no título
    """
    prompt = f"""### Instruction:
Generate a detailed product description based on the following title.

### Input:
{title}

### Response:"""
    
    try:
        # Configura parâmetros de geração
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
        generation_config = {
            'max_new_tokens': max_length,
            'temperature': temperature,
            'do_sample': True,
            'top_p': 0.9,
            'pad_token_id': tokenizer.eos_token_id,
            'repetition_penalty': 1.1
        }
        
        with torch.no_grad():
            outputs = model.generate(**inputs, **generation_config)
        
        # Decodifica apenas a resposta gerada
        response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
        return response.strip()
        
    except Exception as e:
        return f"Erro na geração: {str(e)}"

# Interface interativa simples
print("🎯 Interface de Teste Interativo do Modelo")
print("=" * 50)
print("Digite títulos de produtos para gerar descrições!")
print("(Digite 'sair' para encerrar)")
print("=" * 50)

# Função para teste interativo (adaptada para notebook)
def test_interactive():
    sample_titles = [
        "Wireless Gaming Mouse with RGB",
        "Eco-Friendly Bamboo Phone Case",
        "Premium Leather Wallet for Men",
        "Portable Bluetooth Speaker Waterproof",
        "LED Desk Lamp with USB Charging"
    ]
    
    print("\n🔹 Exemplos de títulos para testar:")
    for i, title in enumerate(sample_titles, 1):
        print(f"  {i}. {title}")
    
    print("\n🎲 Teste automático com exemplos:")
    for title in sample_titles[:3]:  # Testa apenas os 3 primeiros
        print(f"\n📝 Título: {title}")
        print("🤖 Descrição gerada:")
        description = generate_product_description(title)
        print(f"   {description}")
        print("-" * 40)

# Executa teste interativo
test_interactive()

### 7.2 Resumo dos Resultados

#### Conclusões do Tech Challenge

In [ ]:
print("🎯 RESUMO DO TECH CHALLENGE - FINE-TUNING")
print("=" * 60)
print()
print("📊 CONFIGURAÇÃO UTILIZADA:")
print(f"  🤖 Modelo: {CONFIG['model_name']}")
print(f"  📏 Tamanho máximo: {CONFIG['max_seq_length']} tokens")
print(f"  🔧 LoRA r: {CONFIG['lora_r']}, alpha: {CONFIG['lora_alpha']}")
print(f"  📈 Learning rate: {CONFIG['learning_rate']}")
print(f"  🔄 Steps: {CONFIG['max_steps']}")
print()
print("💾 DADOS PROCESSADOS:")
if 'training_data' in locals():
    print(f"  📁 Amostras processadas: {len(training_data)}")
    final_approval_rate = (len(training_data) / CONFIG['sample_size']) * 100
    print(f"  ✅ Taxa de aprovação: {final_approval_rate:.1f}%")
print()
print("🚀 RESULTADOS:")
print("  ✅ Fine-tuning executado com sucesso")
print("  ✅ Modelo otimizado para descrições de produtos Amazon")
print("  ✅ Interface interativa funcional")
print()
print("💡 MELHORIAS IMPLEMENTADAS:")
print("  🔍 Sistema avançado de limpeza de dados")
print("  📊 Filtragem por qualidade de conteúdo")
print("  🎯 Formato instruction-following otimizado")
print("  ⚡ Configuração otimizada para Google Colab")
print()
print("🎉 TECH CHALLENGE CONCLUÍDO COM SUCESSO!")
print("=" * 60)